In [ ]:
#!pip install -q ctransformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00


In [1]:
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML", model_file = 'llama-2-7b-chat.ggmlv3.q2_K.bin', callbacks=[StreamingStdOutCallbackHandler()],temperature=0)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\mm0450.MAVEN\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mm0450.MAVEN\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
template = """
[INST] <<SYS>>
You are a accurate data annotator of Recipe and respond single word without any reasoning. Using the given ingredients of a recipe, You need to just return whether it is 'NonVeg' or 'Veg' or 'Vegan'. Ingredients: 
<</SYS>>
{ingredients}[/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["ingredients"])

In [18]:
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

In [19]:
ingredients = "['1 c. firmly packed brown sugar', '1/2 c. evaporated milk', '1/2 tsp. vanilla', '1/2 c. broken nuts (pecans)', '2 Tbsp. butter or margarine', '3 1/2 c. bite size shredded rice biscuits']"

In [20]:
response = llm_chain.run(ingredients)



> Entering new LLMChain chain...
Prompt after formatting:

[INST] <<SYS>>
You are a accurate data annotator of Recipe and respond single word without any reasoning. Using the given ingredients of a recipe, You need to just return whether it is 'NonVeg' or 'Veg' or 'Vegan'. Ingredients: 
<</SYS>>
['1 c. firmly packed brown sugar', '1/2 c. evaporated milk', '1/2 tsp. vanilla', '1/2 c. broken nuts (pecans)', '2 Tbsp. butter or margarine', '3 1/2 c. bite size shredded rice biscuits'][/INST]

Veg
> Finished chain.


Classification Testing on Data

In [21]:
import pandas as pd

In [22]:
df = pd.read_csv("recipe_top_1000_rows.csv")

In [23]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1000 non-null   int64 
 1   title        1000 non-null   object
 2   ingredients  1000 non-null   object
 3   directions   1000 non-null   object
 4   link         1000 non-null   object
 5   source       1000 non-null   object
 6   NER          1000 non-null   object
 7   site         1000 non-null   object
dtypes: int64(1), object(7)
memory usage: 62.6+ KB


Unnamed: 0                  title   
0           0    No-Bake Nut Cookies  \
1           1  Jewell Ball'S Chicken   
2           2            Creamy Corn   
3           3          Chicken Funny   
4           4   Reeses Cups(Candy)     

                                         ingredients   
0  ["1 c. firmly packed brown sugar", "1/2 c. eva...  \
1  ["1 small jar chipped beef, cut up", "4 boned ...   
2  ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg...   
3  ["1 large whole chicken", "2 (10 1/2 oz.) cans...   
4  ["1 c. peanut butter", "3/4 c. graham cracker ...   

                                          directions   
0  ["In a heavy 2-quart saucepan, mix brown sugar...  \
1  ["Place chipped beef on bottom of baking dish....   
2  ["In a slow cooker, combine all ingredients. C...   
3  ["Boil and debone chicken.", "Put bite size pi...   
4  ["Combine first four ingredients and press in ...   

                                              link    source   
0   www.cookbooks.com/Recipe-Details.aspx?id=44874  Gathered  \
1  www.cookbooks.com/Recipe-Details.aspx?id=699419  Gathered   
2   www.cookbooks.com/Recipe-Details.aspx?id=10570  Gathered   
3  www.cookbooks.com/Recipe-Details.aspx?id=897570  Gathered   
4  www.cookbooks.com/Recipe-Details.aspx?id=659239  Gathered   

                                                 NER               site  
0  ["bite size shredded rice biscuits", "vanilla"...  www.cookbooks.com  
1  ["cream of mushroom soup", "beef", "sour cream...  www.cookbooks.com  
2  ["frozen corn", "pepper", "cream cheese", "gar...  www.cookbooks.com  
3  ["chicken gravy", "cream of mushroom soup", "c...  www.cookbooks.com  
4  ["graham cracker crumbs", "powdered sugar", "p...  www.cookbooks.com

In [24]:
df = df.head(501)

In [25]:
df['LLAMA2_OP'] = df.ingredients.apply(lambda x: llm_chain.run(x))



> Entering new LLMChain chain...
Prompt after formatting:

[INST] <<SYS>>
You are a accurate data annotator of Recipe and respond single word without any reasoning. Using the given ingredients of a recipe, You need to just return whether it is 'NonVeg' or 'Veg' or 'Vegan'. Ingredients: 
<</SYS>>
["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"][/INST]

Veg
> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

[INST] <<SYS>>
You are a accurate data annotator of Recipe and respond single word without any reasoning. Using the given ingredients of a recipe, You need to just return whether it is 'NonVeg' or 'Veg' or 'Vegan'. Ingredients: 
<</SYS>>
["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"][/INST]

NonVeg
> Finished chain.


> Entering new LLMChain chain.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   501 non-null    int64 
 1   title        501 non-null    object
 2   ingredients  501 non-null    object
 3   directions   501 non-null    object
 4   link         501 non-null    object
 5   source       501 non-null    object
 6   NER          501 non-null    object
 7   site         501 non-null    object
 8   LLAMA2_OP    501 non-null    object
dtypes: int64(1), object(8)
memory usage: 35.4+ KB


In [27]:
df.to_csv("llama2_raw_output_v1.csv")